# F_score计算

F-score是Piotroski于2000年提出，探讨了是否简单地使用财务报表信息来做基本面分析，可以明显地改善低估值策略，有效地规避低估值陷阱。

F-score从盈利能力、偿债能力与资本结构、运营能力三个方面提出 9 个指标来衡量公司的基本面状况， 华创证券结合 A 股情况做了细微地修改，主要是将年度改成了 TTM，如下表所示：

![alt](https://image.joinquant.com/bee3c6c93f80644141e2af908ef5c747)

这 9 个指标均为 0-1 变量，将这 9 个指标加起来即为 F-score。Piotroski（2000）依据得分将得分在 0 至 3 之间的股票设置为 Low 组，将得分在 4 至 6 之前的股票设置为 Middle 组，将得分在 7 至 9 之间的股票设置为 High 组。

根据研报回测结果，Middle 组 和 High 组选择的股票基本面更好，大幅度回撤的概率更低。研报指出，F-score 具备一定的收益区分能力，高 F-score 组明显的跑赢低 F-score 组，但研报并不把 F-score 当作一个收益预测因子，而更多的是风控手段。

策略思路是：选择F_score大于等于5且市值最小的5只股票持有，每10天调仓。

In [1]:
from jqdata import *
import pandas as pd

In [2]:
security_list = get_index_stocks('000300.XSHG')
my_watch_date = datetime.date(2022,5,23)
one_year_ago = my_watch_date - datetime.timedelta(days=365)

In [3]:
h = get_history_fundamentals(security_list,
                             [indicator.adjusted_profit,
                              balance.total_current_assets,
                              balance.total_assets,
                              balance.total_current_liability,
                              balance.total_non_current_liability,
                              cash_flow.net_operate_cash_flow,
                              income.operating_revenue,
                              income.operating_cost,
                              ],
                             watch_date=my_watch_date, count=5).dropna()  # 连续的5个季度

In [10]:
def ttm_sum(x):
    return x.iloc[1:].sum()

def ttm_avg(x):
    return x.iloc[1:].mean()

def pre_ttm_sum(x):
    return x.iloc[:-1].sum()

def pre_ttm_avg(x):
    return x.iloc[:-1].mean()

def val_1(x):
    return x.iloc[-1]

def val_2(x):
#     return x.iloc[-2]
    return x.iloc[-4:].sum() if len(x)>4 else np.nan

In [5]:
# 扣非利润
adjusted_profit_ttm = h.groupby('code')['adjusted_profit'].apply(ttm_sum)
adjusted_profit_ttm_pre = h.groupby('code')['adjusted_profit'].apply(pre_ttm_sum)

# 总资产平均
total_assets_avg = h.groupby('code')['total_assets'].apply(ttm_avg)
total_assets_avg_pre = h.groupby('code')['total_assets'].apply(pre_ttm_avg)

# 经营活动产生的现金流量净额
net_operate_cash_flow_ttm = h.groupby('code')['net_operate_cash_flow'].apply(ttm_sum)

# 长期负债率: 长期负债/总资产
long_term_debt_ratio = h.groupby('code')['total_non_current_liability'].apply(val_1) / h.groupby('code')['total_assets'].apply(val_1)
long_term_debt_ratio_pre = h.groupby('code')['total_non_current_liability'].apply(val_2) / h.groupby('code')['total_assets'].apply(val_2)

# 流动比率：流动资产/流动负债
current_ratio = h.groupby('code')['total_current_assets'].apply(val_1) / h.groupby('code')['total_current_liability'].apply(val_1)
current_ratio_pre = h.groupby('code')['total_current_assets'].apply(val_2) / h.groupby('code')['total_current_liability'].apply(val_2)

# 营业收入
operating_revenue_ttm = h.groupby('code')['operating_revenue'].apply(ttm_sum)
operating_revenue_ttm_pre = h.groupby('code')['operating_revenue'].apply(pre_ttm_sum)

# 营业成本
operating_cost_ttm = h.groupby('code')['operating_cost'].apply(ttm_sum)
operating_cost_ttm_pre = h.groupby('code')['operating_cost'].apply(pre_ttm_sum)

In [7]:
# 1. ROA 资产收益率
# 统计口径：扣非净利润_TTM/总资产_AVG
# 计分标准：大于零为1，否则为0
roa = adjusted_profit_ttm / total_assets_avg
roa_pre = adjusted_profit_ttm_pre / total_assets_avg_pre

# 2. OCFOA 经营活动产生的现金流量净额/总资产
# 统计口径：经营活动产生的现金流量净额_TTM/总资产_AVG
# 计分标准：大于零为1，否则为0
ocfoa = net_operate_cash_flow_ttm / total_assets_avg

# 3. ROA_CHG 资产收益率变化
# 统计口径：当期资产收益率 - 上一期资产收益率
# 计分标准：大于零为1，否则为0
roa_chg = roa - roa_pre

# 4. OCFOA_ROA 应计收益率: 经营活动产生的现金流量净额/总资产 -资产收益率
# 统计口径：经营活动产生的现金流量净额比总资产 -资产收益率
# 计分标准：大于零为1，否则为0
ocfoa_roa = ocfoa - roa

# 5. LTDR_CHG 长期负债率变化 (长期负债率=长期负债/总资产)
# 统计口径：当期长期负债率 - 上一期长期负债率
# 计分标准：大于零为0，否则为1
ltdr_chg = long_term_debt_ratio - long_term_debt_ratio_pre

# 6. CR_CHG 流动比率变化 (流动比率=流动资产/流动负债)
# 统计口径：当期流动比率 - 上一期流动比率
# 计分标准：大于零为1，否则为0
cr_chg = current_ratio - current_ratio_pre

# 7. 股票是否增发
# 统计口径：最近一年股票是否增发
# 计分标准：是为0，否则为1
spo_list = list(set(finance.run_query(
    query(
        finance.STK_CAPITAL_CHANGE.code
    ).filter(
        finance.STK_CAPITAL_CHANGE.code.in_(security_list),
        finance.STK_CAPITAL_CHANGE.pub_date.between(one_year_ago, my_watch_date),
        finance.STK_CAPITAL_CHANGE.change_reason_id == 306004)
)['code']))

spo_score = pd.Series(True, index = security_list)
if spo_list:
    spo_score[spo_list] = False
    
# 8. GPM_CHG 毛利率变化 (毛利率=1-营业成本/营业收入)
# 统计口径：当期毛利率_TTM - 上一期毛利率_TTM
# 计分标准：大于零为1，否则为0
gpm_chg = operating_cost_ttm_pre/operating_revenue_ttm_pre - operating_cost_ttm/operating_revenue_ttm

# 9. TAT_CHG 资产周转率变化(资产周转率=营业收入/总资产)
# 统计口径：当期资产周转率_TTM - 上一期资产周转率_TTM
# 计分标准：大于零为1，否则为0
tat_chg = operating_revenue_ttm/total_assets_avg - operating_revenue_ttm_pre/total_assets_avg_pre

In [8]:
# 计算得分total
# 统计口径：这9个指标均为0-1变量，将这9个指标加起来即为F-Score。
# 分组方式：Piotroski（2000）依据得分将得分在0至3之间的股票设置为Low组，4至6设置为Middle组，7至9之间设置为High组
df_scores = pd.DataFrame(index=security_list)
# 1
df_scores['roa'] = roa>0
# 2
df_scores['ocfoa'] = ocfoa>0
# 3
df_scores['roa_chg'] = roa_chg>0
# 4
df_scores['ocfoa_roa'] = ocfoa_roa>0
# 5
df_scores['ltdr_chg'] = ltdr_chg<=0
# 6
df_scores['cr_chg'] = cr_chg>0
# 7
df_scores['spo'] = spo_score
# 8
df_scores['gpm_chg'] = gpm_chg>0
# 9
df_scores['tat_chg'] = tat_chg>0

# 合计
df_scores = df_scores.dropna()
# df_scores['total'] = df_scores['roa'] + df_scores['ocfoa'] + df_scores['roa_chg'] + df_scores['ocfoa_roa'] + df_scores['ltdr_chg'] + df_scores['cr_chg'] + df_scores['spo'] + df_scores['gpm_chg'] + df_scores['tat_chg']
df_scores['total'] = df_scores.sum(axis=1)

In [12]:
df_scores.head()

,roa,ocfoa,roa_chg,ocfoa_roa,ltdr_chg,cr_chg,spo,gpm_chg,tat_chg,total
000002.XSHE,True,False,False,False,False,True,True,False,False,3
000063.XSHE,True,True,True,True,False,True,True,True,False,7
000066.XSHE,True,False,False,False,False,True,True,True,False,4
000069.XSHE,True,True,True,True,True,True,True,False,False,7
000100.XSHE,True,True,True,True,True,False,True,False,True,7
